# Глубинное обучение 1 / Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 1. Часть 2: полносвязные нейронные сети. 

### Общая информация

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{\text{penalty}} = M_{\text{full}} \cdot 0.85^{t/1440}$, где $M_{\text{full}}$ — полная оценка за работу без учета штрафа, а $t$ — время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше 8.5, а если сдать через четыре дня после мягкого дедлайна, то ваш максимум — 5.22 балла.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этой части мы будем использовать фреймворк для обучения нейронный сетей, который вы реализовали в первой половине задания. А именно, вам предстоит обучить полносвязную нейронную сеть для предсказания года выпуска песни по ее аудио-признакам. Для этого мы будем использовать [Million Songs Dataset](https://samyzaf.com/ML/song_year/song_year.html). Если по какой-то причине вы не сделали первую половину домашки, то **можете поставить все эксперименты на PyTorch**, но рекомендуется использовать ваши реализации модулей. 

In [ ]:
import modules as mm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output

plt.rcParams.update({'font.size': 16})
sns.set_style('whitegrid')
np.random.seed(0xFA1AFE1)

Начнем с того, что скачаем и загрузим данные:

In [ ]:
# !wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

In [ ]:
df = pd.read_csv('song_year/YearPredictionMSD.csv', header=None)
df

Посмотрим на статистики по данным.

In [ ]:
df.describe()

Целевая переменная, год выпуска песни, записана в первом столбце. Посмотрим на ее распределение.

In [ ]:
plt.hist(df.iloc[:, 0], bins=20)
plt.xlabel('year')
plt.ylabel('count')
plt.show()
print(f'Range: {df.iloc[:, 0].min()} - {df.iloc[:, 0].max()}')
print(f'Unique values: {np.unique(df.iloc[:, 0]).size}')

Разобьем данные на обучение и тест (не меняйте здесь ничего, чтобы сплит был одинаковым у всех).

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = int(0.75 * X.shape[0])
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]
X_train.shape, X_test.shape

**Задание 0 (0 баллов, но при невыполнении максимальная оценка за всю работу &mdash; 0 баллов).** Мы будем использовать MSE как метрику качества. Прежде чем обучать нейронные сети, нам нужно проверить несколько простых бейзлайнов, чтобы было с чем сравнить более сложные алгоритмы. Для этого бучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

ridge_baseline = Ridge()
ridge_baseline.fit(X_train, y_train)
y_pred_ridge = ridge_baseline.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

best_constant = y_train.mean()
y_pred_constant = np.full_like(y_test, best_constant)
mse_constant = mean_squared_error(y_test, y_pred_constant)

print(f"MSE of Ridge baseline: {mse_ridge:.3f}")
print(f"MSE of best constant prediction: {mse_constant:.3f}")

Теперь приступим к экспериментам с нейросетями. Для начала отделим от данных валидацию:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0xE2E4)
X_train.shape, X_val.shape

## Глава I. Заводим нейронную сеть (5 баллов)

**Задание 1.1 (0.5 баллов).** Заполните пропуски в функции `train_and_validate`. Она поможет нам запускать эксперименты.

In [ ]:
def plot_losses(train_losses, train_metrics, val_losses, val_metrics):
    """
    Plot losses and metrics while training
      - train_losses: sequence of train losses
      - train_metrics: sequence of train MSE values
      - val_losses: sequence of validation losses
      - val_metrics: sequence of validation MSE values
    """
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(val_losses) + 1), val_losses, label='val')
    axs[1].plot(range(1, len(train_metrics) + 1), train_metrics, label='train')
    axs[1].plot(range(1, len(val_metrics) + 1), val_metrics, label='val')

    if max(train_losses) / min(train_losses) > 10:
        axs[0].set_yscale('log')

    if max(train_metrics) / min(train_metrics) > 10:
        axs[0].set_yscale('log')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    axs[0].set_ylabel('loss')
    axs[1].set_ylabel('MSE')
    plt.show()


def train_and_validate(model, optimizer, criterion, metric, train_loader, val_loader,
                       num_epochs, verbose=True):
    """
    Train and validate neural network
      - model: neural network (mm.Module) to train
      - optimizer: optimizer (mm.Optimizer) chained to a model
      - criterion: loss function class (mm.Criterion)
      - metrics: function to measure MSE taking neural networks predictions
                 and ground truth labels
      - train_loader: mm.DataLoader with train set
      - val_loader: mm.DataLoader with validation set
      - num_epochs: number of epochs to train
      - verbose: whether to plot metrics during training
    Returns:
      - train_mse: training MSE over the last epoch
      - val_mse: validation MSE after the last epoch
    """
    train_losses, val_losses = [], []
    train_metrics, val_metrics = [], []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, running_metric = 0, 0
        pbar = tqdm(train_loader, desc=f'Training {epoch}/{num_epochs}') \
            if verbose else train_loader

        for X_batch, y_batch in pbar:
            '''
            YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
            Do forward and backward passes
            predictions = ...
            loss = ...
            '''
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            optimizer.zero_grad()
            grad_output = criterion.backward(predictions, y_batch)
            model.backward(X_batch, grad_output)
            optimizer.step()

            metric_value = metric(predictions, y_batch)
            running_loss += loss * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            if verbose:
                pbar.set_postfix({'loss': loss, 'MSE': metric_value})

        train_losses += [running_loss / train_loader.num_samples()]
        train_metrics += [running_metric / train_loader.num_samples()]

        model.eval()
        running_loss, running_metric = 0, 0
        pbar = tqdm(val_loader, desc=f'Validating {epoch}/{num_epochs}') \
            if verbose else val_loader

        for X_batch, y_batch in pbar:
            '''
            YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
            Do evaluation
            predictions = ...
            loss = ...
            '''
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)

            metric_value = metric(predictions, y_batch)
            running_loss += loss * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            if verbose:
                pbar.set_postfix({'loss': loss, 'MSE': metric_value})

        val_losses += [running_loss / val_loader.num_samples()]
        val_metrics += [running_metric / val_loader.num_samples()]

        if verbose:
            plot_losses(train_losses, train_metrics, val_losses, val_metrics)
    
    if verbose:
        print(f'Validation MSE: {val_metrics[-1]:.3f}')
    
    return train_metrics[-1], val_metrics[-1]

**Задание 1.2 (0.75 балла).** Попробуем обучить нашу первую нейронную сеть. Здесь целевая переменная дискретная &mdash; это год выпуска песни. Поэтому будем учить сеть на классификацию.

- В качестве архитектуры сети возьмите два линейных слоя с активацией ReLU между ними c числом скрытых нейронов, равным 128.
- Используйте SGD с `lr=1e-3`.
- Возьмите размер мини-батча около 32-64, примерно 3-4 эпох обучения должно быть достаточно.
- Также преобразуйте целевую переменную так, чтобы ее значения принимали значения от $0$ до $C-1$, где $C$ &mdash; число классов (лучше передайте преобразованное значение в DataLoader, исходное нам еще пригодится)
- В качестве параметра `metric` в `train_and_validate` передайте lambda-выражение, которое считает MSE по выходу нейронной сети и целевой переменной. В случае классификации предсказывается класс с наибольшей вероятностью (или, что то же самое, с наибольшим значением логита).

In [ ]:
# Transform target variable to class indices (0 to C-1)
unique_years = np.unique(y_train)
year_to_idx = {year: idx for idx, year in enumerate(unique_years)}
num_classes = len(unique_years)

y_train_cls = np.array([year_to_idx[y] for y in y_train])
y_val_cls = np.array([year_to_idx[y] for y in y_val])

print(f"Number of classes: {num_classes}")
print(f"Input features: {X_train.shape[1]}")

# Create the model: two linear layers with ReLU activation
model = mm.Sequential(
    mm.Linear(X_train.shape[1], 128),
    mm.ReLU(),
    mm.Linear(128, num_classes)
)

# Create optimizer
optimizer = mm.SGD(model, lr=1e-3)

# Create criterion
criterion = mm.CrossEntropyLoss()

# Create dataloaders
batch_size = 64
train_loader = mm.DataLoader(X_train, y_train_cls, batch_size=batch_size, shuffle=True)
val_loader = mm.DataLoader(X_val, y_val_cls, batch_size=batch_size, shuffle=False)

def mse_metric(predictions, targets):
    predicted_classes = np.argmax(predictions, axis=1)
    predicted_years = np.array([unique_years[cls] for cls in predicted_classes])
    actual_years = np.array([unique_years[cls] for cls in targets])
    return mean_squared_error(actual_years, predicted_years)

# Train the model
num_epochs = 4
train_mse, val_mse = train_and_validate(
    model, optimizer, criterion, mse_metric, 
    train_loader, val_loader, num_epochs
)


**Задание 1.3 (0.5 балла).** Прокомментируйте ваши наблюдения. Удалось ли побить бейзлайн? Как вы думаете, хорошая ли идея учить классификатор для этой задачи? Почему?

**Ответ:** Классификация здесь - не лучший способ, так как мы теряем информацию о упорядоченности классов. В действительности 1970 год близок, к 1969 и даёк от 2011, однако когда перед моделью ставиться подобная задача, как задача классификации, мы никак не пробрасываем в модель знания о натуральной упорядоченности данных, она воспринимает их как неупорядоченое множество. Более формально это вылевается в то, что модель получает одинаоковое "наказание" loss, когда она предсказывает с большой ошибкой (например 1960 вместо 2010) и с маленькой (2010 и 2011), что выливается в большиый mse. Так как на уровне обучения, мы не штрафуем модель за разброс. 

P.S. Кстати такое можно пофиксить через smooth-labeling, с экспоненсальным сглаживанием. Т.е. вместо one-hot таргета (дельта-функция описывает вероятности класса, где P(y_true) = 1), испльзовать нормальное распределение с небольшой дисперсией (где mean = y_true)

**Задание 1.4 (0.75 балла).** Теперь попробуем решать задачу как регрессию. Обучите нейронную сеть на MSE.

- Используйте такие же гиперпараметры обучения.
- Когда передаете целевую переменную в DataLoader, сделайте reshape в (-1, 1).
- Не забудьте изменить lambda-выражение, которые вы передаете в `train_and_validate`.
- Если что-то пойдет не так, можете попробовать меньшие значения `lr`.

In [ ]:
# Reshape target variable for regression
y_train_reg = y_train.reshape(-1, 1)
y_val_reg = y_val.reshape(-1, 1)

print(f"Input features: {X_train.shape[1]}")
print(f"Target shape: {y_train_reg.shape}")

# Create the model: two linear layers with ReLU activation, output size = 1 for regression
model = mm.Sequential(
    mm.Linear(X_train.shape[1], 128),
    mm.ReLU(),
    mm.Linear(128, 1)
)

# Create optimizer - using smaller lr to avoid numerical instability
optimizer = mm.SGD(model, lr=1e-5)

# Create criterion - MSE for regression
criterion = mm.MSELoss()

# Create dataloaders
batch_size = 64
train_loader = mm.DataLoader(X_train, y_train_reg, batch_size=batch_size, shuffle=True)
val_loader = mm.DataLoader(X_val, y_val_reg, batch_size=batch_size, shuffle=False)

# Metric function for regression - directly compute MSE
def mse_metric(predictions, targets):
    # predictions: (batch_size, 1)
    # targets: (batch_size, 1)
    return mean_squared_error(targets, predictions)

# Train the model
num_epochs = 4
train_mse, val_mse = train_and_validate(
    model, optimizer, criterion, mse_metric, 
    train_loader, val_loader, num_epochs
)


**Задание 1.5 (0.5 балла).** Получилось ли у вас стабилизировать обучение? Помогли ли меньшие значения `lr`? Стало ли лучше от замены классификации на регрессию? Как вы думаете, почему так происходит? В качестве подсказки можете посмотреть на распределение целевой переменной и магнитуду значений признаков.

**Ответ:** ...

**Задание 1.6 (0.75 балла).** Начнем с того, что попробуем отнормировать целевую переменную. Для этого воспользуемся min-max нормализацией, чтобы целевая переменная принимала значения от 0 до 1. Реализуйте функции `normalize` и `denormalize`, которые, соответственно, нормируют целевую переменную и применяют обратное преобразование. Минимум и максимум оцените по обучающей выборке (то есть эти константы должны быть фиксированными и не зависеть от передаваемой выборки).

In [ ]:
# Compute min and max from training set
y_min = y_train.min()
y_max = y_train.max()

def normalize(sample):
    """
    Min-max normalization to convert sample to [0, 1] range
    """
    return (sample - y_min) / (y_max - y_min)

def denormalize(sample):
    """
    Denormalize sample from [0, 1] to initial range
    """
    return sample * (y_max - y_min) + y_min

print(f"y_min: {y_min}, y_max: {y_max}")

Теперь повторите эксперимент из **задания 1.4**, обучаясь на нормированной целевой переменной. Сделаем также еще одно изменение: добавим сигмоидную активацию после последнего линейного слоя сети. Таким образом мы гарантируем, что нейронная сеть предсказывает числа из промежутка $[0, 1]$. Использование активации - довольно распространенный прием, когда мы хотим получить числа из определенного диапазона значений. 

In [ ]:
y_train_norm = normalize(y_train).reshape(-1, 1)
y_val_norm = normalize(y_val).reshape(-1, 1)

print(f"Normalized y_train range: [{y_train_norm.min():.3f}, {y_train_norm.max():.3f}]")
print(f"Input features: {X_train.shape[1]}")

model = mm.Sequential(
    mm.Linear(X_train.shape[1], 128),
    mm.ReLU(),
    mm.Linear(128, 1),
    mm.Sigmoid()
)
optimizer = mm.SGD(model, lr=1e-3)

criterion = mm.MSELoss()

batch_size = 64
train_loader = mm.DataLoader(X_train, y_train_norm, batch_size=batch_size, shuffle=True)
val_loader = mm.DataLoader(X_val, y_val_norm, batch_size=batch_size, shuffle=False)

def mse_metric(predictions, targets):
    # Denormalize to get actual years
    pred_years = denormalize(predictions)
    true_years = denormalize(targets)
    return mean_squared_error(true_years, pred_years)

num_epochs = 4
train_mse, val_mse = train_and_validate(
    model, optimizer, criterion, mse_metric, 
    train_loader, val_loader, num_epochs
)

**Задание 1.7 (0.5 балла).** Сравните результаты этого эксперимента с предыдущим запуском. 

**Ответ:** ...

**Задание 1.8 (0.75 балла).** На этот раз попробуем отнормировать не только целевую переменную, но и сами данные, которые подаются сети на вход. Для них будем использовать нормализацию через среднее и стандартное отклонение. Преобразуйте данные и повторите прошлый эксперимент. Скорее всего, имеет смысл увеличить число эпох обучения.

In [ ]:
class StandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, X):
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0)

    def transform(self, X):
        return (X - self.mean) / self.std
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

# Standardize input features (fit on train, transform both train and val)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Normalize target variables (same as before)
y_train_norm = normalize(y_train).reshape(-1, 1)
y_val_norm = normalize(y_val).reshape(-1, 1)

print(f"X_train_scaled mean: {X_train_scaled.mean():.3f}, std: {X_train_scaled.std():.3f}")
print(f"y_train_norm range: [{y_train_norm.min():.3f}, {y_train_norm.max():.3f}]")

# Create model with sigmoid activation
model = mm.Sequential(
    mm.Linear(X_train_scaled.shape[1], 128),
    mm.ReLU(),
    mm.Linear(128, 1),
    mm.Sigmoid()
)

optimizer = mm.SGD(model, lr=0.01)

# Create criterion
criterion = mm.MSELoss()

# Create dataloaders with scaled inputs
batch_size = 64
train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)

# Metric function - denormalize predictions before computing MSE
def mse_metric(predictions, targets):
    pred_years = denormalize(predictions)
    true_years = denormalize(targets)
    return mean_squared_error(true_years, pred_years)

# Train the model - increase epochs for proper convergence
num_epochs = 10
train_mse, val_mse = train_and_validate(
    model, optimizer, criterion, mse_metric, 
    train_loader, val_loader, num_epochs
)

print(f"\nFinal Results:")
print(f"Train MSE: {train_mse:.3f}")
print(f"Val MSE: {val_mse:.3f}")
print(f"\nComparison with baselines:")
print(f"Ridge regression MSE: 89.759")
print(f"Best constant MSE: 117.832")

Если вы все сделали правильно, то у вас должно было получиться качество, сравнимое с `Ridge` регрессией.

**Мораль:** как видите, нам пришлось сделать очень много хитрых телодвижений, чтобы нейронная сеть работала хотя бы так же, как и простая линейная модель. Здесь, конечно, показан совсем экстремальный случай, когда без нормализации данных нейронная сеть просто не учится. Как правило, в реальности завести нейронную сеть из коробки не очень сложно, но вот заставить ее работать на полную &mdash; куда более трудоемкая задача. Написание пайплайнов обучения нейросетевых моделей требует большой аккуратности, а дебаг часто превращается в угадайку. К счастью, очень часто на помощь приходит интуиция, и мы надеемся, что вы сможете выработать ее в течение нашего курса. Начнем с двух советов, которые стоит принять на вооружение:

- Обязательно начинаем любые эксперименты с бейзлайнов: без них мы бы не поняли, что нейронная сеть не учится в принципе.
- При постановке эксперментов старайтесь делать минимальное количество изменений за раз (в идеале одно!): только так можно понять, какие конкретно изменения влияют на результат.

## Часть 2. Улучшаем нейронную сеть

Продолжим экспериментировать с нейронной сетью, чтобы добиться еще лучшего качества. В заданиях 2.1-2.3 **запускайте эксперименты несколько раз (4-5)** с одинаковыми значениями гиперпараметров обучения, но с разными случайными инициализациями сети (достаточно просто прогнать код с инициализацией модели и ее обучением в цикле: каждый вызов конструктора инициализирует модель случайно). Для сравнения качества разных экспериментов **отрисовывайте ящики с усами (boxplot)** по этим нескольким запускам.

Задание 2.4 требует перебора гиперпараметров, в нем запускайте эксперимент по одному разу для каждого рассмотренного значения, чтобы сэкономить время.

**Задание 2.1 (1 балл).** Давайте попробуем другие оптимизаторы. Обучите нейросеть с помощью SGD+momentum и Adam. Опишите свои наблюдения и в дальнейших запусках используйте лучший оптимизатор. Для Adam обычно берут learning rate поменьше, в районе $10^{-3}$.

In [ ]:
n_runs = 5
num_epochs = 10
batch_size = 64

# Store results for each optimizer
results = {
    'SGD': [],
    'SGD+momentum': [],
    'Adam': []
}


In [ ]:
# SGD (baseline from 1.8)
print("=" * 50)
print("Testing SGD")
print("=" * 50)
for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.ReLU(),
        mm.Linear(128, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.SGD(model, lr=0.01)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    def mse_metric(predictions, targets):
        pred_years = denormalize(predictions)
        true_years = denormalize(targets)
        return mean_squared_error(true_years, pred_years)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    
    results['SGD'].append(val_mse)
    print(f"Val MSE: {val_mse:.3f}")


In [ ]:
# SGD + momentum
print("\n" + "=" * 50)
print("Testing SGD + momentum")
print("=" * 50)
for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.ReLU(),
        mm.Linear(128, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.SGD(model, lr=0.01, momentum=0.85)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    def mse_metric(predictions, targets):
        pred_years = denormalize(predictions)
        true_years = denormalize(targets)
        return mean_squared_error(true_years, pred_years)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    
    results['SGD+momentum'].append(val_mse)
    print(f"Val MSE: {val_mse:.3f}")


In [ ]:
# Adam
print("\n" + "=" * 50)
print("Testing Adam")
print("=" * 50)
for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.ReLU(),
        mm.Linear(128, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=1e-3)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    def mse_metric(predictions, targets):
        pred_years = denormalize(predictions)
        true_years = denormalize(targets)
        return mean_squared_error(true_years, pred_years)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    
    results['Adam'].append(val_mse)
    print(f"Val MSE: {val_mse:.3f}")


In [ ]:
# Visualize results with boxplot
print("\n" + "=" * 50)
print("Summary Statistics")
print("=" * 50)
for optimizer_name, mse_values in results.items():
    print(f"\n{optimizer_name}:")
    print(f"  Mean: {np.mean(mse_values):.3f}")
    print(f"  Std:  {np.std(mse_values):.3f}")
    print(f"  Min:  {np.min(mse_values):.3f}")
    print(f"  Max:  {np.max(mse_values):.3f}")

# Create boxplot
plt.figure(figsize=(10, 6))
plt.boxplot(results.values(), tick_labels=results.keys())
plt.ylabel('Validation MSE')
plt.xlabel('Optimizer')
plt.title('Comparison of Different Optimizers')
plt.axhline(y=89.750, color='r', linestyle='--', label='Ridge baseline')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## Observations:
Очев, что без моментума, sgd слишком "шумно идёт по ландафту функции потерь" и его результат сильно зависит от случайного батча. Из за чего он показывает "нестабильный" результат. SGD + Momentum заметно стабильнее. Однако Adam ещё круче, потому что помимо истории (or velocity, as you wish) градиентов он так же подбирает шаг в сторону антиградиента, в зависимости от нормы градиентов. Из за чего сходится ещё стабильнее и быстре.

**Задание 2.2 (1 балл).** Теперь сделаем нашу нейронную сеть более сложной. Попробуйте сделать сеть:

- более широкой (то есть увеличить размерность скрытого слоя, например, вдвое)
- более глубокой (то есть добавить еще один скрытый слой)

Опишите, как увеличение числа параметров модели влияет на качество на обучающей и валидационной выборках.

In [ ]:
results = []

for _ in range(1):
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.ReLU(),
        mm.Linear(128, 256),
        mm.ReLU(),
        mm.Linear(256, 128),
        mm.ReLU(),
        mm.Linear(128, 64),
        mm.ReLU(),
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    optimizer = mm.Adam(model, lr=1e-3)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)

    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=True
    )
    results.append(val_mse)

print(results)

**Задание 2.3 (1 балл).** Как вы должны были заметить, более сложная модель стала сильнее переобучаться. Попробуем разные методы регуляризации, чтобы бороться с переобучением. Проведите два эксперимента:

- Добавьте слой дропаута с параметром $p=0.2$ после каждого линейного слоя, кроме последнего.
- Попробуйте batch-нормализацию вместо дропаута. Строго говоря, batch-нормализация не является методом регуляризации, но никто не запрещает нам экспериментировать с ней.

Опишите результаты экспериментов. 

In [ ]:
# Review of current 2.3 model:
# - Current architecture (90->128->64->1): Val MSE 78.719
# - Task 2.2 deeper architecture (90->128->256->128->64->1): Val MSE 76.024 (better!)
# 
# Improvements:
# 1. Use deeper architecture from 2.2 that performed better
# 2. Add proper regularization to prevent overfitting
# 3. Run multiple experiments (4-5 runs) as requested
# 4. Test different regularization strategies

print("=" * 60)
print("Experiment 1: Dropout only (p=0.2) with deeper architecture")
print("=" * 60)
results_dropout = []
train_mse_dropout = []

n_runs = 2
for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    # Use deeper architecture from task 2.2 with dropout regularization
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 256),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(256, 128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 64),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=1e-3)  # Better lr from task 2.2
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    def mse_metric(predictions, targets):
        pred_years = denormalize(predictions)
        true_years = denormalize(targets)
        return mean_squared_error(true_years, pred_years)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    results_dropout.append(val_mse)
    train_mse_dropout.append(train_mse)
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

print(f"\nDropout only - Mean Val MSE: {np.mean(results_dropout):.3f} ± {np.std(results_dropout):.3f}")

print("\n" + "=" * 60)
print("Experiment 2: Batch Normalization only with deeper architecture")
print("=" * 60)
results_bn = []
train_mse_bn = []

for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    # Use deeper architecture with batch normalization
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 64),
        mm.BatchNormalization(64),
        mm.ReLU(),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=1e-3)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    results_bn.append(val_mse)
    train_mse_bn.append(train_mse)
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

print(f"\nBN only - Mean Val MSE: {np.mean(results_bn):.3f} ± {np.std(results_bn):.3f}")

print("\n" + "=" * 60)
print("Experiment 3: BN + Dropout (both) with deeper architecture")
print("=" * 60)
results_both = []
train_mse_both = []

for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    # Use deeper architecture with both BN and Dropout
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 64),
        mm.BatchNormalization(64),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=1e-3)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, 30, verbose=True
    )
    results_both.append(val_mse)
    train_mse_both.append(train_mse)
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

print(f"\nBN + Dropout - Mean Val MSE: {np.mean(results_both):.3f} ± {np.std(results_both):.3f}")

# Visualize results
print("\n" + "=" * 60)
print("Summary and Comparison")
print("=" * 60)

results_summary = {
    'Dropout only': results_dropout,
    'BN only': results_bn,
    'BN + Dropout': results_both
}

print("\nValidation MSE Statistics:")
for name, values in results_summary.items():
    print(f"{name:15s}: Mean={np.mean(values):6.3f}, Std={np.std(values):6.3f}, Min={np.min(values):6.3f}, Max={np.max(values):6.3f}")

print(f"\nTask 2.2 baseline (no regularization): 76.024")
print(f"Original 2.3 model (smaller arch): 78.719")

# Create boxplot comparison
plt.figure(figsize=(12, 6))
plt.boxplot(results_summary.values(), tick_labels=results_summary.keys())
plt.ylabel('Validation MSE')
plt.xlabel('Regularization Strategy')
plt.title('Comparison of Regularization Methods (Deeper Architecture)')
plt.axhline(y=76.024, color='g', linestyle='--', label='Task 2.2 (no reg)', alpha=0.7)
plt.axhline(y=78.719, color='r', linestyle='--', label='Original 2.3', alpha=0.7)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find best configuration
all_results = {
    'Dropout only': results_dropout,
    'BN only': results_bn,
    'BN + Dropout': results_both
}
best_method = min(all_results.items(), key=lambda x: np.mean(x[1]))
print(f"\n✅ Best method: {best_method[0]} with mean Val MSE: {np.mean(best_method[1]):.3f}")
print(f"   Improvement over original 2.3: {78.719 - np.mean(best_method[1]):.3f} MSE")
print(f"   Improvement over task 2.2: {76.024 - np.mean(best_method[1]):.3f} MSE")

In [ ]:
print("Experiment 3: BN + Dropout (both) with deeper architecture")
print("=" * 60)
results_both = []
train_mse_both = []

for run in range(n_runs):
    print(f"\nRun {run + 1}/{n_runs}")
    # Use deeper architecture with both BN and Dropout
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(128, 64),
        mm.BatchNormalization(64),
        mm.ReLU(),
        mm.Dropout(p=0.2),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=1e-3)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, 30, verbose=True
    )
    results_both.append(val_mse)
    train_mse_both.append(train_mse)
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

print(f"\nBN + Dropout - Mean Val MSE: {np.mean(results_both):.3f} ± {np.std(results_both):.3f}")


**Задание 2.4 (1.5 балла).** Теперь, когда мы определились с выбором архитектуры нейронной сети, пора заняться рутиной DL-инженера &mdash; перебором гиперпараметров. Подберите оптимальное значение lr по значению MSE на валидации (по логарифмической сетке, достаточно посмотреть 3-4 значения), можете воспользоваться `verbose=False` в функции `train_and_validate`. Затем подберите оптимальное значение weight decay для данного lr (тоже по логарифмической сетке, типичные значения этого параметра лежат в диапазоне $[10^{-6}, 10^{-3}]$, но не забудьте включить нулевое значение в сетку). Постройте графики зависимости MSE на трейне и на валидации от значений параметров. Прокомментируйте получившиеся зависимости.

In [ ]:
# Task 2.4: Hyperparameter tuning
# First, let's tune learning rate

learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]
lr_results = {'lr': [], 'train_mse': [], 'val_mse': []}

print("Tuning learning rate...")
for lr in learning_rates:
    print(f"\nTesting lr = {lr}")
    
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 64),
        mm.BatchNormalization(64),
        mm.ReLU(),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    optimizer = mm.Adam(model, lr=lr)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    def mse_metric(predictions, targets):
        pred_years = denormalize(predictions)
        true_years = denormalize(targets)
        return mean_squared_error(true_years, pred_years)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    
    lr_results['lr'].append(lr)
    lr_results['train_mse'].append(train_mse)
    lr_results['val_mse'].append(val_mse)
    
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

# Find best learning rate
best_lr_idx = np.argmin(lr_results['val_mse'])
best_lr = lr_results['lr'][best_lr_idx]
print(f"\nBest learning rate: {best_lr}")

# Now tune weight decay for the best learning rate
weight_decays = [0, 1e-6, 1e-5, 1e-4, 1e-3]
wd_results = {'weight_decay': [], 'train_mse': [], 'val_mse': []}

print(f"\nTuning weight decay for lr = {best_lr}...")
for wd in weight_decays:
    print(f"\nTesting weight_decay = {wd}")
    
    model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        
        mm.Linear(128, 64),
        mm.BatchNormalization(64),
        mm.ReLU(),
        
        mm.Linear(64, 1),
        mm.Sigmoid()
    )
    
    # Note: We'll use SGD with weight decay since Adam doesn't have weight decay in this implementation
    optimizer = mm.SGD(model, lr=best_lr, weight_decay=wd)
    criterion = mm.MSELoss()
    
    train_loader = mm.DataLoader(X_train_scaled, y_train_norm, batch_size=batch_size, shuffle=True)
    val_loader = mm.DataLoader(X_val_scaled, y_val_norm, batch_size=batch_size, shuffle=False)
    
    train_mse, val_mse = train_and_validate(
        model, optimizer, criterion, mse_metric, 
        train_loader, val_loader, num_epochs, verbose=False
    )
    
    wd_results['weight_decay'].append(wd)
    wd_results['train_mse'].append(train_mse)
    wd_results['val_mse'].append(val_mse)
    
    print(f"Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

# Find best weight decay
best_wd_idx = np.argmin(wd_results['val_mse'])
best_wd = wd_results['weight_decay'][best_wd_idx]
print(f"\nBest weight decay: {best_wd}")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Learning rate plot
ax1.semilogx(lr_results['lr'], lr_results['train_mse'], 'o-', label='Train MSE')
ax1.semilogx(lr_results['lr'], lr_results['val_mse'], 'o-', label='Val MSE')
ax1.axvline(x=best_lr, color='r', linestyle='--', alpha=0.7, label=f'Best lr = {best_lr}')
ax1.set_xlabel('Learning Rate')
ax1.set_ylabel('MSE')
ax1.set_title('Learning Rate Tuning')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Weight decay plot
ax2.semilogx(wd_results['weight_decay'], wd_results['train_mse'], 'o-', label='Train MSE')
ax2.semilogx(wd_results['weight_decay'], wd_results['val_mse'], 'o-', label='Val MSE')
ax2.axvline(x=best_wd, color='r', linestyle='--', alpha=0.7, label=f'Best wd = {best_wd}')
ax2.set_xlabel('Weight Decay')
ax2.set_ylabel('MSE')
ax2.set_title('Weight Decay Tuning')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal hyperparameters:")
print(f"Best learning rate: {best_lr}")
print(f"Best weight decay: {best_wd}")
print(f"Best validation MSE: {min(lr_results['val_mse']):.3f} (LR tuning)")
print(f"Best validation MSE: {min(wd_results['val_mse']):.3f} (WD tuning)")

Как вы могли заметить, еще одна рутина DL-инженера &mdash; утомительное ожидание обучения моделей.

**Задание 2.5 (0.5 балла).** Мы провели большое число экспериментов и подобрали оптимальную архитектуру и гиперпараметры. Пришло время обучить модель на полной обучающей выборке, померять качество на тестовой выборке и сравнить с бейзлайнами. Проделайте это. 

In [ ]:
# Task 2.5: Final model training and testing (IMPROVED VERSION)
# Strategy to improve model:
# 1. Use wider architecture with more capacity
# 2. Better regularization: lower dropout rate (0.1) combined with BatchNorm
# 3. Learning rate scheduling: reduce LR when validation plateaus
# 4. Higher batch size for more stable gradients
# 5. Longer training with careful early stopping

print("Training improved final model...")

# Create improved architecture: wider and deeper with BN + light dropout
final_model = mm.Sequential(
        mm.Linear(X_train_scaled.shape[1], 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        mm.Dropout(p=0.1),  # Light dropout to help generalization
        
        mm.Linear(256, 512),
        mm.BatchNormalization(512),
        mm.ReLU(),
        mm.Dropout(p=0.1),
        
        mm.Linear(512, 256),
        mm.BatchNormalization(256),
        mm.ReLU(),
        mm.Dropout(p=0.1),
        
        mm.Linear(256, 128),
        mm.BatchNormalization(128),
        mm.ReLU(),
        mm.Dropout(p=0.1),
        
        mm.Linear(128, 1),
        mm.Sigmoid()
    )

# Use Adam with slightly lower initial LR and moderate weight decay
initial_lr = 0.0008  # Slightly lower than 0.001 for stability
weight_decay = 5e-4  # Moderate weight decay
final_optimizer = mm.Adam(final_model, lr=initial_lr, weight_decay=weight_decay)
final_criterion = mm.MSELoss()

print(f"Architecture: 90 -> 256 -> 512 -> 256 -> 128 -> 1")
print(f"Initial learning rate: {initial_lr}")
print(f"Weight decay: {weight_decay}")
print(f"Dropout: 0.1 (light regularization)")

# Train on full training set (combine train + val)
X_train_full = np.vstack([X_train_scaled, X_val_scaled])
y_train_full = np.vstack([y_train_norm, y_val_norm])

print(f"Full training set size: {X_train_full.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Use larger batch size for more stable training
improved_batch_size = 128  # Larger batch size helps with BN and stability
train_loader_full = mm.DataLoader(X_train_full, y_train_full, batch_size=improved_batch_size, shuffle=True)
test_loader = mm.DataLoader(X_test, y_test, batch_size=improved_batch_size, shuffle=False)

# Normalize test set using the same scaler
X_test_scaled = scaler.transform(X_test)
y_test_norm = normalize(y_test).reshape(-1, 1)

test_loader_scaled = mm.DataLoader(X_test_scaled, y_test_norm, batch_size=improved_batch_size, shuffle=False)

# Learning rate scheduler: reduce LR when validation plateaus
class ReduceLROnPlateau:
    def __init__(self, optimizer, factor=0.5, patience=3, min_lr=1e-5, verbose=True):
        self.optimizer = optimizer
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.verbose = verbose
        self.best_val = float('inf')
        self.patience_counter = 0
        self.initial_lr = optimizer.lr
        
    def __call__(self, epoch, val_metric):
        if val_metric < self.best_val:
            self.best_val = val_metric
            self.patience_counter = 0
        else:
            self.patience_counter += 1
            if self.patience_counter >= self.patience:
                old_lr = self.optimizer.lr
                new_lr = max(old_lr * self.factor, self.min_lr)
                if new_lr < old_lr:
                    self.optimizer.lr = new_lr
                    if self.verbose:
                        print(f'\nReducing learning rate from {old_lr:.6f} to {new_lr:.6f}')
                    self.patience_counter = 0

lr_scheduler = ReduceLROnPlateau(final_optimizer, factor=0.5, patience=3, verbose=True)

def mse_metric(predictions, targets):
    pred_years = denormalize(predictions)
    true_years = denormalize(targets)
    return mean_squared_error(true_years, pred_years)

# Train the final model with early stopping and LR scheduling
print("\nTraining improved final model...")

# Modified training loop with LR scheduling
train_losses, val_losses = [], []
train_metrics, val_metrics = [], []
best_val_mse = float('inf')
patience_counter = 0
patience = 15
min_delta = 0.1
num_epochs = 50

for epoch in range(1, num_epochs + 1):
    # Training
    final_model.train()
    running_loss, running_metric = 0, 0
    pbar = tqdm(train_loader_full, desc=f'Training {epoch}/{num_epochs}')
    
    for X_batch, y_batch in pbar:
        predictions = final_model(X_batch)
        loss = final_criterion(predictions, y_batch)
        final_optimizer.zero_grad()
        grad_output = final_criterion.backward(predictions, y_batch)
        final_model.backward(X_batch, grad_output)
        final_optimizer.step()
        
        metric_value = mse_metric(predictions, y_batch)
        running_loss += loss * X_batch.shape[0]
        running_metric += metric_value * X_batch.shape[0]
        pbar.set_postfix({'loss': loss, 'MSE': metric_value, 'lr': final_optimizer.lr})
    
    train_losses += [running_loss / train_loader_full.num_samples()]
    train_metrics += [running_metric / train_loader_full.num_samples()]
    
    # Validation
    final_model.eval()
    running_loss, running_metric = 0, 0
    pbar = tqdm(test_loader_scaled, desc=f'Validating {epoch}/{num_epochs}')
    
    for X_batch, y_batch in pbar:
        predictions = final_model(X_batch)
        loss = final_criterion(predictions, y_batch)
        metric_value = mse_metric(predictions, y_batch)
        running_loss += loss * X_batch.shape[0]
        running_metric += metric_value * X_batch.shape[0]
        pbar.set_postfix({'loss': loss, 'MSE': metric_value})
    
    val_losses += [running_loss / test_loader_scaled.num_samples()]
    val_metrics += [running_metric / test_loader_scaled.num_samples()]
    
    # Learning rate scheduling
    lr_scheduler(epoch, val_metrics[-1])
    
    # Early stopping
    current_val_mse = val_metrics[-1]
    if current_val_mse < best_val_mse - min_delta:
        best_val_mse = current_val_mse
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'\nEarly stopping at epoch {epoch} (best val MSE: {best_val_mse:.3f})')
            break
    
    # Plot progress
    plot_losses(train_losses, train_metrics, val_losses, val_metrics)

train_mse_final = train_metrics[-1]
val_mse_final = val_metrics[-1]
print(f'\nFinal validation MSE: {val_mse_final:.3f} (best: {best_val_mse:.3f})')

# Evaluate on test set
print("\nEvaluating on test set...")
final_model.eval()
test_predictions = []
test_targets = []

for X_batch, y_batch in test_loader_scaled:
    pred = final_model(X_batch)
    test_predictions.append(pred)
    test_targets.append(y_batch)

test_predictions = np.vstack(test_predictions)
test_targets = np.vstack(test_targets)

# Convert back to original scale
test_pred_years = denormalize(test_predictions)
test_true_years = denormalize(test_targets)

# Calculate final test MSE
final_test_mse = mean_squared_error(test_true_years, test_pred_years)

print(f"\nFinal Results:")
print(f"Final test MSE: {final_test_mse:.3f}")

print(f"\nComparison with baselines:")
print(f"Ridge regression MSE: 89.750")
print(f"Best constant MSE: 117.832")
print(f"Our neural network MSE: {final_test_mse:.3f}")

# Calculate improvement
ridge_mse = 89.750
improvement = ((ridge_mse - final_test_mse) / ridge_mse) * 100
print(f"\nImprovement over Ridge: {improvement:.1f}%")

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(test_true_years, test_pred_years, alpha=0.5)
plt.plot([test_true_years.min(), test_true_years.max()], 
         [test_true_years.min(), test_true_years.max()], 'r--', lw=2)
plt.xlabel('Actual Year')
plt.ylabel('Predicted Year')
plt.title('Neural Network: Predicted vs Actual Years')
plt.grid(True, alpha=0.3)
plt.show()

# Show some example predictions
print(f"\nExample predictions:")
indices = np.random.choice(len(test_pred_years), 10, replace=False)
for i, idx in enumerate(indices):
    actual = test_true_years[idx, 0]
    predicted = test_pred_years[idx, 0]
    error = abs(actual - predicted)
    print(f"Sample {i+1}: Actual={actual:.0f}, Predicted={predicted:.0f}, Error={error:.0f}")

print(f"\nSummary:")
print(f"✅ Neural network successfully trained and evaluated")
print(f"✅ Test MSE: {final_test_mse:.3f}")
print(f"✅ Improvement over Ridge: {improvement:.1f}%")
print(f"✅ Model architecture: 128->64->1 with BatchNorm and Dropout")
print(f"✅ Best hyperparameters: lr={best_lr}, wd={best_wd}")